In [ ]:
# Notebook imported libraries
import pandas as pd  
import numpy as np
import requests 
import re
import io
import datetime
import tabula
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from tabula import read_pdf
from datetime import datetime
import calplot

### Webscraping unnamed table data
http://floridalottery.com/exptkt/ff.htm

In [ ]:
# Fantasy5
url = ('http://floridalottery.com/exptkt/ff.htm') 

### Request object

In [ ]:
r = requests.get(url)
print(r.status_code) # 200 Good to Go!

### Soup object

In [ ]:
soup = BeautifulSoup(r.text.strip(), 'html.parser') 

### Parse entire html page, all tables!

In [ ]:
# Return all table data
table_data = []

for node in soup.find_all('td'): 
    table_data.append(''.join(node.find_all(text=True)))

### Remove empty unwanted spaces from list.

In [ ]:
while("" in table_data):
    table_data.remove("")

### Remove ("-") values from lists.

In [ ]:
while("-" in table_data):
    table_data.remove("-")

### List to ndarray

In [ ]:
arr = np.array(table_data)
print(arr.shape)

### Converting 1D array to 2D array.

In [ ]:
# len of array / num of cols
x = int(len(arr)/6)
arr2 = np.reshape(arr, (int(x), 6))# Fantasy Five
print(arr2)

In [ ]:
arr2.shape
print(type(arr2))

### Convert remaining table_data to pandas DataFrame

In [ ]:
Fantasy_Five = pd.DataFrame(arr2, columns= ['Date', 'bal1', 
                                            'bal2', 'bal3',
                                            'bal4', 'bal5'])
Fantasy_Five.head()

### Convert remaining strings to pandas DateTime object<br>
if not datetime object drop row in Date column, and set Date column as index.

In [ ]:
Fantasy_Five['Date'] = pd.to_datetime(Fantasy_Five['Date'], 
                                      format='%m/%d/%y', 
                                      errors= 'coerce')
Fantasy_Five = Fantasy_Five.set_index('Date')
Fantasy_Five.head()

In [ ]:
# Remove Date NaT values by the row
Fantasy_Five = Fantasy_Five.loc[Fantasy_Five.index.notnull()]
Fantasy_Five.head()

In [ ]:
# Object Dtypes
Fantasy_Five.info()

### Cast all columns to int32

In [ ]:
# Chaning object type to int32
Fantasy_Five = Fantasy_Five.astype({
    'bal1': 'int32',
    'bal2': 'int32',
    'bal3': 'int32',
    'bal4': 'int32',
    'bal5': 'int32',
})
Fantasy_Five.info()

In [ ]:
# calendar plot
calplot.calplot(Fantasy_Five['bal1'], yearascending= False, cmap = 'YlGn')

### Cast all DataFrames to .csv or .json files

In [ ]:
# Create json, csv files
Fantasy_Five.to_csv('FantasyFive_csv.csv') 
Fantasy_Five.to_json('FantasyFive_json.json')

In [ ]:
# Read csv or json file
Fantasy5_Prediction = pd.read_csv('FantasyFive_csv', index_col= 'Date')
Fantasy5_Prediction.head()

### Tabula to read Winners PDF

In [ ]:
# read pdf
mylist = tabula.read_pdf("f5of36winners_Dec09_to_present.pdf")

In [ ]:
# convert PDF into CSV file
tabula.convert_into("f5of36winners_Dec09_to_present.pdf"
                    "output.csv", output_format="csv")

### Winners Data only goes back to the year 2012

### EDA 